In [1]:
#Import the required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Conv1D, GlobalMaxPooling1D, Dense, Dropout , LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, GRU, Conv1D, MaxPooling1D, Dropout, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from tensorflow.keras.optimizers import legacy


In [2]:
print(tf.test.gpu_device_name())


In [3]:
imdb = pd.read_csv("/content/IMDB Dataset.csv")

In [4]:
imdb

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
imdb.drop_duplicates(inplace=True)

In [6]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
texts = imdb['review'].values
labels = imdb['sentiment'].values

In [8]:
# Preprocess the reviews
stop_words = set(stopwords.words('english'))
processed_texts = []

for text in texts:
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters, URLs, and email addresses
    text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Convert to lower case and tokenize
    text = word_tokenize(text.lower())

    # Remove stop words
    text = [word for word in text if not word in stop_words]

    # Remove repeated characters and single characters
    processed_text = []

    for word in text:
        word = re.sub(r'\b(\w*?)(\w)\2+(\w*?\b|\b)', r'\1\2\3', word)
        if len(word) > 1:
            processed_text.append(word)
    processed_texts.append(' '.join(processed_text))


In [9]:
len(processed_texts[1])

638

In [23]:
max_words = 10000
max_length = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(processed_texts)
sequences = tokenizer.texts_to_sequences(processed_texts)
X = pad_sequences(sequences, maxlen=max_length)
y = np.array(imdb['sentiment'].map({'positive': 1, 'negative': 0}))

In [24]:
X.shape

(49582, 500)

In [25]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
with tf.device('/GPU:0'):

    # Build the neural network
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=500),
        #GRU(10, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        #Bidirectional(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        #Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])

# Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Train the model
    early_stopping = EarlyStopping(patience=3)
    model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1, callbacks=[early_stopping])
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 conv1d_6 (Conv1D)           (None, 496, 128)          82048     
                                                                 
 conv1d_7 (Conv1D)           (None, 492, 128)          82048     
                                                                 
 global_max_pooling1d_3 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                      

In [28]:
with tf.device('/GPU:0'):

    # Build the neural network
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=500),
        #GRU(10, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        #Bidirectional(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        #Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])

# Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Train the model
    early_stopping = EarlyStopping(patience=3)
    model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.1, callbacks=[early_stopping])
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 conv1d_12 (Conv1D)          (None, 496, 128)          82048     
                                                                 
 conv1d_13 (Conv1D)          (None, 492, 128)          82048     
                                                                 
 conv1d_14 (Conv1D)          (None, 488, 128)          82048     
                                                                 
 conv1d_15 (Conv1D)          (None, 484, 128)          82048     
                                                                 
 global_max_pooling1d_5 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                      

In [29]:
with tf.device('/GPU:0'):

    # Build the neural network
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=500),
        GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        #Bidirectional(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        #Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])

# Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Train the model
    early_stopping = EarlyStopping(patience=3)
    model.fit(X_train, y_train, epochs=2, batch_size=128, validation_split=0.1, callbacks=[early_stopping])
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 gru (GRU)                   (None, 500, 128)          99072     
                                                                 
 conv1d_16 (Conv1D)          (None, 496, 128)          82048     
                                                                 
 conv1d_17 (Conv1D)          (None, 492, 128)          82048     
                                                                 
 global_max_pooling1d_6 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_12 (Dense)            (None, 128)               16512     
                                                      